In [1]:
import pywinauto
from pywinauto import clipboard
import time

In [133]:
app = pywinauto.Application().connect(path=r"C:\同花顺软件\同花顺免费模拟炒股软件\weituo\模拟炒股\xiadan.exe",timeout=10)

c:\users\kalen\appdata\local\programs\python\python37\lib\site-packages\pywinauto\application.py:978: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  UserWarning)


In [135]:
main = app.top_window()

In [136]:
main.window_text()

'网上股票交易系统5.0'

In [137]:
main.handle

460504

In [6]:
def get_left_menus_handle():
    while True:
        try:
            handle = main.window(
                control_id=0x81,
#                 handle=0x20EB0,
                class_name='SysTreeView32'
            )
            # sometime can't find handle ready, must retry
            handle.wait('ready', 2)
            return handle
        except:
            pass

In [7]:
get_left_menus_handle()

## 获取资金余额

In [21]:
get_left_menus_handle().get_item([4]).click()

In [25]:
result = {
    "资金余额":  main.window(control_id=0x40E, class_name='Static').window_text()
}

In [26]:
result

{'资金余额': '49661.64'}

## 查询持有股票

In [67]:
get_left_menus_handle().get_item([0]).click()

In [140]:
switch_tab = main.window(
    control_id=0x54B,
    class_name='CCustomTabCtrl'
)
time.sleep(0.5)
main.set_focus()
time.sleep(0.5)
pywinauto.keyboard.SendKeys('^w')

grid = main.window(control_id=0x417, class_name='CVirtualGridCtrl')

grid.set_focus() #.right_click()   # 模拟右键

pywinauto.keyboard.SendKeys('^c')
# time.sleep(0.1)
# pywinauto.keyboard.SendKeys('{DOWN}')
# time.sleep(0.1)
# pywinauto.keyboard.SendKeys("{ENTER}")


print(clipboard.GetData())  # 

证券代码	证券名称	股票余额	可用余额	冻结数量	盈亏	成本价	盈亏比(%)	市价	市值	交易市场	股东帐户	实际数量	可申赎数量	
000333	美的集团	200	200	0	6157.880	17.811	172.87	48.600	9720.000	深圳Ａ股	00101106569	200	200	
000651	格力电器	1100	1100	0	4060.700	46.608	7.92	50.300	55330.000	深圳Ａ股	00101106569	1100	1100	
162411	华宝油气	1300	1200	100	-473.980	0.591	-61.73	0.226	293.800	深圳Ａ股	00101106569	1300	1200	
600048	保利地产	3800	3800	0	2718.060	13.285	5.38	14.000	53200.000	上海Ａ股	A476826692	3800	3800	
601155	新城控股	1600	1600	0	-3270.520	29.284	-6.98	27.240	43584.000	上海Ａ股	A476826692	1600	1600


In [141]:
import pandas as pd
import io

In [142]:
data = clipboard.GetData()
df = pd.read_csv(io.StringIO(data),
                 delimiter='\t',
                 na_filter=False)
df

,证券代码,证券名称,股票余额,可用余额,冻结数量,盈亏,成本价,盈亏比(%),市价,市值,交易市场,股东帐户,实际数量,可申赎数量,Unnamed: 14
0,333,美的集团,200,200,0,6157.88,17.811,172.87,48.600,9720.0,深圳Ａ股,00101106569,200,200,
1,651,格力电器,1100,1100,0,4060.70,46.608,7.92,50.300,55330.0,深圳Ａ股,00101106569,1100,1100,
2,162411,华宝油气,1300,1200,100,-473.98,0.591,-61.73,0.226,293.8,深圳Ａ股,00101106569,1300,1200,
3,600048,保利地产,3800,3800,0,2718.06,13.285,5.38,14.000,53200.0,上海Ａ股,A476826692,3800,3800,
4,601155,新城控股,1600,1600,0,-3270.52,29.284,-6.98,27.240,43584.0,上海Ａ股,A476826692,1600,1600,


In [143]:
df.to_dict('records')

[{'证券代码': 333,
  '证券名称': '美的集团',
  '股票余额': 200,
  '可用余额': 200,
  '冻结数量': 0,
  '盈亏': 6157.88,
  '成本价': 17.811,
  '盈亏比(%)': 172.87,
  '市价': 48.6,
  '市值': 9720.0,
  '交易市场': '深圳Ａ股',
  '股东帐户': '00101106569',
  '实际数量': 200,
  '可申赎数量': 200,
  'Unnamed: 14': ''},
 {'证券代码': 651,
  '证券名称': '格力电器',
  '股票余额': 1100,
  '可用余额': 1100,
  '冻结数量': 0,
  '盈亏': 4060.7,
  '成本价': 46.608000000000004,
  '盈亏比(%)': 7.92,
  '市价': 50.3,
  '市值': 55330.0,
  '交易市场': '深圳Ａ股',
  '股东帐户': '00101106569',
  '实际数量': 1100,
  '可申赎数量': 1100,
  'Unnamed: 14': ''},
 {'证券代码': 162411,
  '证券名称': '华宝油气',
  '股票余额': 1300,
  '可用余额': 1200,
  '冻结数量': 100,
  '盈亏': -473.98,
  '成本价': 0.591,
  '盈亏比(%)': -61.73,
  '市价': 0.226,
  '市值': 293.8,
  '交易市场': '深圳Ａ股',
  '股东帐户': '00101106569',
  '实际数量': 1300,
  '可申赎数量': 1200,
  'Unnamed: 14': ''},
 {'证券代码': 600048,
  '证券名称': '保利地产',
  '股票余额': 3800,
  '可用余额': 3800,
  '冻结数量': 0,
  '盈亏': 2718.06,
  '成本价': 13.285,
  '盈亏比(%)': 5.38,
  '市价': 14.0,
  '市值': 53200.0,
  '交易市场': '上海Ａ股',
  '股东帐户': 'A476826692',
  '实

In [36]:
df["市值"].sum()

162127.8

## 模拟买入

In [37]:
get_left_menus_handle().get_item(['买入[F1]']).click()

In [38]:
grid = main.window(control_id=0x417, class_name='CVirtualGridCtrl')

In [39]:
stock_id = "162411"

In [49]:
main.window(control_id=0x408, class_name="Edit").set_text(stock_id)  # 设置股票代码

main.window(control_id=0x409, class_name="Edit").set_text("0.218")  # 设置价格

main.window(control_id=0x40A, class_name="Edit").set_text("100")   # 设置股数目

main.window(control_id=0x3EE, class_name="Button").click()   # 点击买入
time.sleep(0.5)
app.top_window().set_focus()
app.top_window().window(control_id=0x6, class_name='Button').click()  # 确定买入
time.sleep(0.5)
app.top_window().set_focus()
print(app.top_window().window(control_id=0x3EC, class_name='Static').window_text())
time.sleep(0.5)
app.top_window().set_focus()
app.top_window().window(control_id=0x2, class_name='Button').click()  # 确定

您的买入委托已成功提交，合同编号：1428398429。


## 模拟卖出

In [51]:
get_left_menus_handle().get_item(['卖出[F2]']).click()

In [52]:
main.window(control_id=0x408, class_name="Edit").set_text(stock_id)  # 设置股票代码

main.window(control_id=0x409, class_name="Edit").set_text("0.238")  # 设置价格

main.window(control_id=0x40A, class_name="Edit").set_text("100")   # 设置股数目

main.window(control_id=0x3EE, class_name="Button").click()   # 点击卖出
time.sleep(0.5)
app.top_window().set_focus()
app.top_window().window(control_id=0x6, class_name='Button').click()  # 确定卖出
time.sleep(0.5)
app.top_window().set_focus()
print(app.top_window().window(control_id=0x3EC, class_name='Static').window_text())
time.sleep(0.5)
app.top_window().set_focus()
app.top_window().window(control_id=0x2, class_name='Button').click()  # 确定

您的卖出委托已成功提交，合同编号：1428473361。


## 获取当前的委托

In [71]:
get_left_menus_handle().get_item([0]).click()

In [80]:
switch_tab = main.window(
    control_id=0x54B,
    class_name='CCustomTabCtrl'
)
main.set_focus()
time.sleep(0.5)
pywinauto.keyboard.SendKeys('^r')

grid = main.window(control_id=0x417, class_name='CVirtualGridCtrl')

grid.set_focus() #.right_click()   # 模拟右键

pywinauto.keyboard.SendKeys('^c')

data = clipboard.GetData() #
print(data)

委托时间	证券代码	证券名称	操作	备注	委托数量	成交数量	撤消数量	委托价格	订单类型	成交均价	合同编号	
16:53:01	162411	华宝油气	买入	全部撤单	100	0	100	0.218	限价	0.000	1428348934	
16:54:30	162411	华宝油气	买入	全部撤单	100	0	100	0.218	限价	0.000	1428398429	
17:00:25	162411	华宝油气	卖出	未成交	100	0	0	0.238	限价	0.000	1428473361	
17:06:34	162411	华宝油气	买入	未成交	100	0	0	0.226	限价	0.000	1428426470	
16:52:04	162411	华宝油气	买入	全部撤单	100	0	100	0.218	限价	0.000	1428339249


In [81]:
df = pd.read_csv(io.StringIO(data),
                 delimiter='\t',
                 na_filter=False)
df

,委托时间,证券代码,证券名称,操作,备注,委托数量,成交数量,撤消数量,委托价格,订单类型,成交均价,合同编号,Unnamed: 12
0,16:53:01,162411,华宝油气,买入,全部撤单,100,0,100,0.218,限价,0.0,1428348934,
1,16:54:30,162411,华宝油气,买入,全部撤单,100,0,100,0.218,限价,0.0,1428398429,
2,17:00:25,162411,华宝油气,卖出,未成交,100,0,0,0.238,限价,0.0,1428473361,
3,17:06:34,162411,华宝油气,买入,未成交,100,0,0,0.226,限价,0.0,1428426470,
4,16:52:04,162411,华宝油气,买入,全部撤单,100,0,100,0.218,限价,0.0,1428339249,


## 获取当日成交

In [82]:
get_left_menus_handle().get_item([0]).click()

In [132]:
switch_tab = main.window(
    control_id=0x54B,
    class_name='CCustomTabCtrl'
)
main.set_focus()
time.sleep(0.5)
pywinauto.keyboard.SendKeys('^e')

grid = main.window(control_id=0x417, class_name='CVirtualGridCtrl')

grid.set_focus() #.right_click()   # 模拟右键

pywinauto.keyboard.SendKeys('^c')

data = clipboard.GetData() #
print(data)
df = pd.read_csv(io.StringIO(data),
                 delimiter='\t',
                 na_filter=False)
df

成交时间	证券代码	证券名称	操作	成交数量	成交均价	成交金额	合同编号	成交编号	撤单数量	股票余额


,成交时间,证券代码,证券名称,操作,成交数量,成交均价,成交金额,合同编号,成交编号,撤单数量,股票余额


## 撤单

In [87]:
get_left_menus_handle().get_item([0]).click()

In [89]:
switch_tab = main.window(
    control_id=0x54B,
    class_name='CCustomTabCtrl'
)
time.sleep(0.5)
main.set_focus()
time.sleep(0.5)
pywinauto.keyboard.SendKeys('^r')

In [114]:
def get_entrusts():
    grid = main.window(
        control_id=0x417,
        class_name='CVirtualGridCtrl'
    )
    grid.set_focus()
    time.sleep(0.5)
    pywinauto.keyboard.SendKeys('^c')
    time.sleep(0.5)

    data = clipboard.GetData()
    print(data)
    df = pd.read_csv(io.StringIO(data),
                     delimiter='\t',
                     na_filter=False)
    return df.to_dict('records')

entrusts = get_entrusts()

委托时间	证券代码	证券名称	操作	备注	委托数量	成交数量	撤消数量	委托价格	订单类型	成交均价	合同编号	
16:53:01	162411	华宝油气	买入	全部撤单	100	0	100	0.218	限价	0.000	1428348934	
16:54:30	162411	华宝油气	买入	全部撤单	100	0	100	0.218	限价	0.000	1428398429	
17:00:25	162411	华宝油气	卖出	未成交	100	0	0	0.238	限价	0.000	1428473361	
17:06:34	162411	华宝油气	买入	未成交	100	0	0	0.226	限价	0.000	1428426470	
17:27:00	162411	华宝油气	买入	未成交	100	0	0	0.226	限价	0.000	1428521310	
17:27:15	162411	华宝油气	买入	未成交	100	0	0	0.226	限价	0.000	1428534004	
16:52:04	162411	华宝油气	买入	全部撤单	100	0	100	0.218	限价	0.000	1428339249


In [115]:
entrusts

[{'委托时间': '16:53:01',
  '证券代码': 162411,
  '证券名称': '华宝油气',
  '操作': '买入',
  '备注': '全部撤单',
  '委托数量': 100,
  '成交数量': 0,
  '撤消数量': 100,
  '委托价格': 0.218,
  '订单类型': '限价',
  '成交均价': 0.0,
  '合同编号': 1428348934,
  'Unnamed: 12': ''},
 {'委托时间': '16:54:30',
  '证券代码': 162411,
  '证券名称': '华宝油气',
  '操作': '买入',
  '备注': '全部撤单',
  '委托数量': 100,
  '成交数量': 0,
  '撤消数量': 100,
  '委托价格': 0.218,
  '订单类型': '限价',
  '成交均价': 0.0,
  '合同编号': 1428398429,
  'Unnamed: 12': ''},
 {'委托时间': '17:00:25',
  '证券代码': 162411,
  '证券名称': '华宝油气',
  '操作': '卖出',
  '备注': '未成交',
  '委托数量': 100,
  '成交数量': 0,
  '撤消数量': 0,
  '委托价格': 0.23800000000000002,
  '订单类型': '限价',
  '成交均价': 0.0,
  '合同编号': 1428473361,
  'Unnamed: 12': ''},
 {'委托时间': '17:06:34',
  '证券代码': 162411,
  '证券名称': '华宝油气',
  '操作': '买入',
  '备注': '未成交',
  '委托数量': 100,
  '成交数量': 0,
  '撤消数量': 0,
  '委托价格': 0.226,
  '订单类型': '限价',
  '成交均价': 0.0,
  '合同编号': 1428426470,
  'Unnamed: 12': ''},
 {'委托时间': '17:27:00',
  '证券代码': 162411,
  '证券名称': '华宝油气',
  '操作': '买入',
  '备注': '未成交',
  '委托数量': 100

In [116]:
entrust_no = "1428473361"

In [117]:
def cancel_by_double_click(row):
    app.top_window().set_focus()
    x = 50
    y = 50 + 20 * row
    app.top_window().window(
            control_id=0x417,
            class_name='CVirtualGridCtrl'
    ).double_click(coords=(x, y))
    
#     app.top_window().window(control_id=0x6, class_name='Button').click()  # 确定撤单

In [125]:
for i, entrust in enumerate(entrusts):
    print(i, entrust["合同编号"])
#     if str(entrust["合同编号"]) == entrust_no:
    if i == 0:
        print("canceling", entrust_no)
        cancel_by_double_click(i)

0 1428348934
canceling 1428473361
1 1428398429
2 1428473361
3 1428426470
4 1428521310
5 1428534004
6 1428339249
